In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
# Install necessary libraries for transformers and NLP
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
!pip install -q transformers datasets torch accelerate
!pip install transformers datasets torch accelerate
!pip install pandas scikit-learn scipy


In [2]:
import pandas as pd

# Load the provided datasets
train_path = "/kaggle/input/datascienee/train (15).csv"
test_path = "/kaggle/input/datascienee/test (15).csv"
#submission_path = "/mnt/data/SampleSubmission (13).csv"

# Read the datasets
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
#submission_df = pd.read_csv(submission_path)

# Display the first few rows of each dataset to understand the structure
train_df.head(), test_df.head()


(      id                                          title  \
 0  11098                            Getting of the weed   
 1    116  Seeking advice on how to face a hospital stay   
 2   7189              Trying to re-enter the work place   
 3   4350                       Family history of cancer   
 4   9749     Inappropriate relationship with therapist.   
 
                                              content  \
 0  [Post removed at request of member]\nHi, welco...   
 1  Hi NMTB,\nThanks for your post. \nI think a lo...   
 2  Hello, \nI’m Cas and for a fair while now I ha...   
 3  Hey everyone.\nI've just discovered that anoth...   
 4  Hi everyone\nI guess the title says it all rea...   
 
                             target  
 0  suicidal-thoughts-and-self-harm  
 1                          anxiety  
 2                          anxiety  
 3                          anxiety  
 4                       depression  ,
       id                                              title  \
 

In [3]:
# Fill missing values
train_df["content"].fillna(train_df["title"], inplace=True)
test_df["content"].fillna(test_df["title"], inplace=True)

<ipython-input-3-b0df48d5ac81>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df["content"].fillna(train_df["title"], inplace=True)
<ipython-input-3-b0df48d5ac81>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

In [4]:
# Text preprocessing function
def preprocess_text(text):
    if pd.isnull(text):
        return ""
    text = text.lower()
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

# Apply preprocessing
train_df["clean_content"] = train_df["content"].apply(preprocess_text)
test_df["clean_content"] = test_df["content"].apply(preprocess_text)

In [5]:
# Reduce TF-IDF feature size for memory efficiency
vectorizer = TfidfVectorizer(max_features=20000)
tfidf_train = vectorizer.fit_transform(train_df["clean_content"])
tfidf_test = vectorizer.transform(test_df["clean_content"])

# Convert to sparse format to save memory
tfidf_train = csr_matrix(tfidf_train)
tfidf_test = csr_matrix(tfidf_test)

In [6]:
# Compute Cosine Similarity in Batches
batch_size = 500
predicted_targets = []

for i in range(0, tfidf_test.shape[0], batch_size):
    batch_test = tfidf_test[i:i+batch_size]
    similarity_scores = cosine_similarity(batch_test, tfidf_train)
    most_similar_indices = similarity_scores.argmax(axis=1)
    batch_predictions = train_df.iloc[most_similar_indices]["target"].values
    predicted_targets.extend(batch_predictions)

# Assign predictions to test dataframe
test_df["predicted_target"] = predicted_targets

# Save the predicted results
test_df[["id", "predicted_target"]].to_csv("enova_1_.csv", index=False)
print("Predicted targets saved to 'predicted_test_targets.csv'.")

Predicted targets saved to 'predicted_test_targets.csv'.


In [7]:
test_df

,id,title,content,clean_content,predicted_target
0,3639,Tailgated,Hello. I was badly tailgated on a busy highway...,hello i was badly tailgated on a busy highway ...,anxiety
1,21493,I am a model and because of mean things my mot...,Morning Scapegoated.\nOk. Scape goated is defi...,morning scapegoated ok scape goated is defined...,depression
2,21215,On-and-off nothingness. I don’t know any more.,"Hi. \nAs the title says, I really just don’t k...",hi as the title says i really just don t know ...,depression
3,13466,Feeling really scared,Hello I am new to all this online forums and s...,hello i am new to all this online forums and s...,depression
4,14084,Looking for people who understand,Hey all.. I never thought I'd be looking for h...,hey all i never thought i d be looking for hel...,anxiety
...,...,...,...,...,...
2457,15718,Stress,Any tips on how to manage the stress that come...,any tips on how to manage the stress that come...,anxiety
2458,889,"30, happily married, beautiful daughter, but s...","Hi all, new member here...struggling at the mo...",hi all new member here struggling at the momen...,depression
2459,2238,Dealing with Guilt and Separation,"Hey everyone, I've been really struggling with...",hey everyone i ve been really struggling with ...,depression
2460,21587,Married but feel alone.,Hi\nI have been married for over 12 years. We ...,hi i have been married for over 12 years we ha...,depression


In [8]:
test_df.to_csv("test_.csv", index=False)